In [1]:
!pip install pymupdf
!pip install tabula-py

     |████████████████████████████████| 5.6MB 1.5MB/s 
     |████████████████████████████████| 10.4MB 9.7MB/s 


In [0]:
import fitz
import tabula
import json
from itertools import groupby
invoices = ['1567166185337-F215839_WITH MARKS.PDF']

In [0]:
def get_invoice_header():
  keys = ['Kund nr. / Costomer no','Datum /Date', 'Faktura nr. / Invoice no','Vår referens / Our reference', 'Vårt order nr. / Our order no', 'Ert order nr. / Your order no', 'Er referens / Your reference', 'Förfallodatum / Due date','Transportsätt / Mode of transport', 'Ert VAT nr. / Your VAT no.', 'Vikt/Weight', 'Leveransvillkor / Terms of delivery', 'Leverans datum / Date of delivery', 'Betalningsvillkor / Terms of payment', 'Godsmärkning / Goods marking']
  values = []
  regions = []
  for key in keys:
    areas = inv.searchFor(key)
    #print (areas[0])
    region = [areas[0][0],areas[0][3]+2,areas[0][2]+10,areas[0][3]+20]
    regions.append(region)
    values.append([w[4:8] for w in words if fitz.Rect(w[:4]) .intersects(fitz.Rect(region))])

  #print(values)
  add_keys = ['Godsmottagare / Consignee', 'Köpare / Buyer']
  for i,key in enumerate(add_keys):
    keys.append(key)
    areas = inv.searchFor(key)
    region = [areas[0][0],areas[0][3]+2,areas[0][2]+80,areas[0][3]+70]
    regions.append(region)
    values.append([w[4:8] for w in words if fitz.Rect(w[:4]) .intersects(fitz.Rect(region))])

  #print(keys)
  #print(values[-1]) 

  # word and line processing
  data=[]
  for i in range(len(values)):
    lines=[]
    for key, group in groupby(values[i], lambda x: x[2]):
      line = " ".join([word[0] for word in group])
      lines.append(line)
    data.append(lines)

  #print(data)

  invoice_header = dict(zip(keys, data))
  #invoice_header = {'invoice_header' : invoice_header}
  #print (invoice_header)
  return invoice_header

def get_invoice_items_tabula():
  # tables are auto detected or required area may be specified by area parameter.
  df = tabula.read_pdf(invoice_file, pages='all')
  items = df.to_dict('records')
  #items = {'Items': items}
  return items

In [4]:
for count,invoice_file in enumerate(invoices):
  invoice = fitz.open(invoice_file)
  
  #print ('Number of pages:',invoice.pageCount)
  # assumed single page
  #print(invoice.metadata)
  
  inv = invoice.loadPage(0)
  
  #text = inv.getText("json")
    
  words = inv.getTextWords()
  '''
  words obtained as a list [x0, y0, x1, y1, word, bno, lno, wno]
  The first 4 entries are the word's rectangle coordinates, the last 3 are
  technical info (block number, line number, word number)
  '''
  # get the invoice header
  invoice_header = get_invoice_header()
  # get the items - 
  invoice_items = get_invoice_items_tabula()
  
  
  invoice_dict = {'invoice_header' : invoice_header,'Items': invoice_items }
  print (invoice_dict)
  
  
  with open('invoice{}.json'.format(count), 'w') as json_file:
    json.dump(invoice_dict, json_file)
  
  
  

{'invoice_header': {'Kund nr. / Costomer no': ['1003'], 'Datum /Date': ['19-08-30'], 'Faktura nr. / Invoice no': ['F215839'], 'Vår referens / Our reference': ['Huvudkontor'], 'Vårt order nr. / Our order no': ['K220199'], 'Ert order nr. / Your order no': ['32156884-002'], 'Er referens / Your reference': ['THORD ELD'], 'Förfallodatum / Due date': ['19-10-29'], 'Transportsätt / Mode of transport': ['DSV Inrikes Styckegods'], 'Ert VAT nr. / Your VAT no.': [], 'Vikt/Weight': ['1332kg'], 'Leveransvillkor / Terms of delivery': ['Fraktfritt'], 'Leverans datum / Date of delivery': ['19-08-30'], 'Betalningsvillkor / Terms of payment': ['60 dagar netto'], 'Godsmärkning / Goods marking': ['32156884'], 'Godsmottagare / Consignee': ['AHLSELL AB', 'TERMINALGATAN 3', 'LOSSNING KL.07.00', '694 81 HALLSBERG'], 'Köpare / Buyer': ['AHLSELL AB', '694 81 HALLSBERG']}, 'Items': [{'Art-nr / E-nr': '176Z2', 'Benämning': 'Väggkonsol', 'Antal': 240.0, 'Enhet': 'Styck', 'A-pris': '206,00', '%': '40,00', 'Belopp':